In [ ]:
# Imports
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fnmatch import fnmatch
from pathlib import Path
from skimage.measure import label
from skimage.measure import regionprops_table, regionprops
from skimage.filters import threshold_otsu
from skimage.io import imread, imshow
from skimage.segmentation import relabel_sequential
from skimage.morphology import binary_closing, binary_dilation, remove_small_objects
from skimage.transform import resize
from sklearn.neighbors import KDTree
from scipy.spatial import cKDTree
from stardist import fill_label_holes, random_label_cmap
import math
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imageio as iio
from tqdm import tqdm
import cv2
import tifffile as tfile

In [ ]:
np.random.seed(42)
lbl_cmap = random_label_cmap()

In [ ]:
def mask_probability_image(prob_img, mask):
    if prob_img.shape != mask.shape:
        print("Masks have different shape.")
        mask = resize(mask, prob_img.shape, preserve_range=True)
    assert prob_img.shape == mask.shape
    
    return prob_img * mask / 255


def threshold_probability_image(prob_img, thresh_C1=0.4):
   
    return np.stack((prob_img[:, :, :] >= thresh_C1), axis=0)


def create_label_imgs(bin_img):
    label_imgs = []
    ch1 = bin_img[:,  :, :]


    
    for frame in range(ch1.shape[0]):
        label_img = label(ch1[frame, :, :], background=0, connectivity=1)
        #label_img = remove_small_objects(label_img, 5)
        label_img = fill_label_holes(label_img)
        label_imgs.append(label_img)
        
    lbl = np.array(label_imgs)

    label_imgs = []

    return lbl


def get_label_props(lbl):
    label_props = {}
    props = {}
    for frame in range(lbl.shape[0]):
        props_C1 = regionprops_table(lbl[frame, :, :], properties=("label", "bbox", "centroid", "area"), 
                                     spacing = (0.085, 0.085)) #spacing = (0.085, 0.085)
        props = {}
        props["C1"] = props_C1
        label_props[frame] = props

    return label_props

## Fiji Macro creates the following file structure:
- Probability Images (C1)
- ROI
- Masks
- Original
- Output_FIJI

## Please put in your parameters here:

In [ ]:
filepath = r"" # Filepath where OUTPUT of Fiji macro is saved

# Channel 1 
thresh_C1 = 0.5 # This is the threshold for the probability maps. 
filter_minsize_C1 = 0.01 # size filter for punctae in um^2
filter_maxsize_C1 = 5 # size filter for punctae in um^2

#Saving options. CAREFUL, setting saving options to "True" will increase the running duration a lot (2-3 hours for full dataset)!
save_segmentation = False
save_segmentation_filtered = False
save_overlayNN = False

## Start of the analysis:

In [ ]:
# Load wound position file

ROIs = []
roi_folder = filepath + "/FijiOutput"
pattern = "*ROICoordinates.csv"

for name in os.listdir(roi_folder):
    if fnmatch(name, pattern):
        #print(name)
        rois = pd.read_csv(filepath + "/FijiOutput/" + name, delimiter=",")
        rois["Identifier"] = name.split("ROICoordinates.csv")[0][:40]
        rois["Circle"] = np.arange(0, rois.shape[0], 1)
        ROIs.append(rois)

df_ROIs = pd.concat(ROIs)

In [ ]:
# Open all Masks
masks = []
mask_folder = filepath + "/FijiOutput"
pattern = "*_Mask.tiff"

for name in os.listdir(mask_folder):
    if fnmatch(name, pattern):
        #print(name)
        mask = imread(os.path.join(mask_folder, name))
        mask = mask[:, :, :]
        #mask = np.moveaxis(mask, 0, 1)     
        masks.append(mask)

In [ ]:
# Open all images
imgs, identifiers = [], []
probability_folder = filepath + "/C1"
pattern = "*.tif"


for name in os.listdir(probability_folder):
    if fnmatch(name, pattern):
        prob = imread(os.path.join(probability_folder, name))
        imgs.append(prob)
        identifiers.append(name.split("C1-")[-1][:40]) #.split(".tif")[0]

In [ ]:
## Check if all data is there
assert len(masks) == len(imgs)

### 2. Data analysis

In [ ]:
# Mask and theshold your image to create binary images
binary_imgs = []
for i in tqdm(range(len(imgs))):
    prob_img = mask_probability_image(imgs[i], masks[i])
    bin_img = threshold_probability_image(prob_img, thresh_C1)
    binary_imgs.append(bin_img)

In [ ]:
lbl_imgs = []

for i in tqdm(range(len(binary_imgs))):
    lbl_img = create_label_imgs(binary_imgs[i])
    lbl_imgs.append(lbl_img)

In [ ]:
#Save segmentation images

if save_segmentation:
    print("Saving label images.")
    for number, lbl_image in tqdm(enumerate(lbl_imgs)):
        newpath = filepath + f"/{identifiers[number]}_Labels"
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        tiffname = newpath + f"/_label.tiff"
        tfile.imwrite(tiffname, lbl_image[:, :, :])

In [ ]:
lbl_props = []

for i in tqdm(range(len(lbl_imgs))):
    lbl_prop = get_label_props(lbl_imgs[i])
    lbl_props.append(lbl_prop)

In [ ]:
list_lbl_props_C1 = []
   
for i in range(len(lbl_props)):
    dfs_C1, dfs_C2 = [], []
    for frame in range(len(lbl_props[i])):
        df_C1_ = pd.DataFrame.from_dict(lbl_props[i][frame]["C1"])
        df_C1_["frame"]= frame+1
        dfs_C1.append(df_C1_)


    df_C1 = pd.concat(dfs_C1, axis=0, ignore_index=True)
    df_C1["Identifier"] = identifiers[i]
    df_C1["Wound_coord_X"] = ROIs[i].iloc[0, 3]
    df_C1["Wound_coord_Y"] = ROIs[i].iloc[0, 4]


    list_lbl_props_C1.append(df_C1)


df_lbl_props_C1 = pd.concat(list_lbl_props_C1, axis=0, ignore_index=True)

In [ ]:
df_lbl_props_C1.to_csv(filepath+"\\"+"C1_punctaedata.csv")

In [ ]:
# Plot punctae sizes - Histogram - Channel 1
plt.figure(figsize=(12, 6))
g = sns.displot(df_lbl_props_C1, x="area", row="Identifier", height=4)
g.refline(x=filter_minsize_C1, color="red")
g.refline(x=filter_maxsize_C1, color="red")
g.set(xlim=(0,1))
g.set(ylim=(0, 500))

In [ ]:
df_lbl_props_C1_backup = df_lbl_props_C1.copy()

### Calculation of wound distance and circle number

In [ ]:
assert df_lbl_props_C1.Identifier.iloc[0] == df_ROIs.Identifier.iloc[0]

In [ ]:
df_lbl_props_C1["Distance"] = (np.sqrt((df_lbl_props_C1["centroid-1"]-df_lbl_props_C1["Wound_coord_X"])**2 + (df_lbl_props_C1["centroid-0"]-df_lbl_props_C1["Wound_coord_Y"])**2))
df_lbl_props_C1["X_Norm"] = abs(df_lbl_props_C1["centroid-1"]-df_lbl_props_C1["Wound_coord_X"])
df_lbl_props_C1["Y_Norm"] = abs(df_lbl_props_C1["centroid-0"]-df_lbl_props_C1["Wound_coord_Y"])

In [ ]:
# Circles
circle_bins = [0, 0.85, 10.85, 20.85, 30.85, 40.85, 50.85, 60.85, 70.85]#[0, 117, 234, 351, 468, 585, 702] # Check how big circles were in original paper
circles = list(range(0,len(circle_bins)-1))
df_lbl_props_C1["Circle"] = pd.cut(df_lbl_props_C1["Distance"], bins = circle_bins, include_lowest=True, labels=circles)

In [ ]:
df_lbl_props_C1.dropna(inplace=True)

In [ ]:
df_lbl_props_C1 = pd.merge(df_lbl_props_C1, df_ROIs.loc[:, ["Area", "Identifier", "Circle"]], left_on= ["Identifier", "Circle"], right_on=["Identifier", "Circle"])

In [ ]:
#Filtering of punctae with size filter
df_lbl_props_C1 = df_lbl_props_C1[df_lbl_props_C1["area"] > filter_minsize_C1]
df_lbl_props_C1 = df_lbl_props_C1[df_lbl_props_C1["area"] < filter_maxsize_C1]

In [ ]:
if save_segmentation_filtered:
    print("Creating filtered label images.")


    lbl_imgs_filtered = []

    for i in tqdm(range(len(identifiers))):

        mask_array = np.full(lbl_imgs[i].shape, False)
        for frame in df_lbl_props_C1[(df_lbl_props_C1.Identifier == identifiers[i])].frame:
            list_to_keep = list(df_lbl_props_C1[((df_lbl_props_C1.Identifier == identifiers[i]) & (df_lbl_props_C1.frame == frame))].label)
            mask = np.isin(lbl_imgs[i][0, frame-1, :, :], list_to_keep)
            mask_array[frame-1, :, :] = mask

    
        result_image = np.copy(lbl_imgs[i])
        result_image[~mask_array] = 0
        lbl_imgs_filtered.append(result_image)

In [ ]:
if save_segmentation_filtered:
    print("Creating filtered label images.")

    #Save filtered images
    for number, lbl_image in tqdm(enumerate(lbl_imgs_filtered)):
        newpath = filepath + f"/{identifiers[number]}_Labels_filtered"
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        tiffname = newpath + f"/filtered_label.tiff"
        tfile.imwrite(tiffname, lbl_image[:, :, :, :])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,20), sharex=True)
plt.tight_layout()
sns.countplot(data=df_lbl_props_C1, x="Identifier", width=0.8, ax=axes).set_title("Channel 1")
                         
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_lbl_props_C1.groupby("Identifier").label.count()

In [ ]:
# Backup of tables
df_C1 = df_lbl_props_C1.copy()

In [ ]:
df_pivot_C1 = df_C1.pivot_table(index=["Identifier", "Circle"], columns="frame", values="label", aggfunc='count', dropna=False)

In [ ]:
df_pivot_C1.loc[:, 1].fillna(value=1, inplace=True, axis=0)    
df_pivot_C1.fillna(value=0, inplace=True, axis=0)

In [ ]:
df_pivot_C1_area = df_C1.pivot_table(index=["Identifier", "Circle"], columns="frame", values="Area", aggfunc='mean', dropna=False)

In [ ]:
fill_value = 2.283
df_pivot_C1_area.loc[(slice(None), 0), :] = df_pivot_C1_area.loc[(slice(None), 0), :].fillna(fill_value)

In [ ]:
df_pivot_C1_area.fillna(method="bfill", inplace=True, axis=1)
df_pivot_C1_area.fillna(method="ffill", inplace=True, axis=1)

In [ ]:
mean_frame_values_C1 = df_pivot_C1.groupby(level="Circle", axis=0).mean()

In [ ]:
# Normalization to circle areas
df_pivot_C1_norm = df_pivot_C1/df_pivot_C1_area

mean_frame_values_C1_norm = df_pivot_C1_norm.groupby(level="Circle", axis=0).mean()

# Normalilzation to first frame
df_pivot_C1_norm_2 = df_pivot_C1_norm.div(df_pivot_C1_norm.iloc[:, 0], axis=0)


In [ ]:
mean_frame_values_C1_norm_2 = df_pivot_C1_norm_2.groupby(level="Circle", axis=0).mean()

In [ ]:
# Plotting
# We start at 0, if you want to plot frame 2 , use 1
min_frame = 0
max_frame = 99
colormap = "inferno" # Other options: "viridis", "inferno", "plasma", "magma", "cividis"
xlabel = "Time"
ylabel = "Circle"
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

# cbar=False
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9,5), gridspec_kw={'height_ratios': [0.8, 0.1]} ) 

sns.heatmap(mean_frame_values_C1.iloc[1:, min_frame:max_frame], vmin=0, vmax=math.ceil(mean_frame_values_C1.max().max()), cmap=colormap, ax=axes[0]).set_title("Clathrin")
sns.heatmap(mean_frame_values_C1.iloc[1:, min_frame:max_frame].mean().to_numpy()[np.newaxis, :], vmin=0, cmap=colormap, ax=axes[1]).set_title("Clathrin_WholeCell")

axes[0].set_xlabel(xlabel)
axes[1].set_xlabel(xlabel)
axes[0].set_ylabel(ylabel)

fig.suptitle("Mean Particle Counts/um^2")
plt.tight_layout(h_pad=5)

# Saving figure
plt.savefig(filepath+"\\"+"ParcticleCounts_NotNormalized.tiff", dpi=600, transparent=True, bbox_inches="tight", format="tiff")

In [ ]:
# Plotting
# We start at 0, if you want to plot frame 2 , use 1
min_frame = 0
max_frame = 99
colormap = "inferno" # Other options: "viridis", "inferno", "plasma", "magma", "cividis"
xlabel = "Time"
ylabel = "Circle"
legendlabel = "Normalized events of TfR and Amph1 after wounding"
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

# cbar=False
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9,5), gridspec_kw={'height_ratios': [0.8, 0.1]} ) 

sns.heatmap(mean_frame_values_C1_norm.iloc[1:, min_frame:max_frame], vmin=0, cmap=colormap, ax=axes[0]).set_title("Clathrin")
sns.heatmap(mean_frame_values_C1_norm.iloc[1:, min_frame:max_frame].mean().to_numpy()[np.newaxis, :], vmin=0, cmap=colormap, ax=axes[1]).set_title("Clathrin_WholeCell")

axes[0].set_xlabel(xlabel)
axes[1].set_xlabel(xlabel)
axes[0].set_ylabel(ylabel)

fig.suptitle("Mean Particle Counts/um^2")
plt.tight_layout(h_pad=5)

# Saving figure
plt.savefig(filepath+"\\"+"ParcticleCounts_Normalized_Circle.tiff", dpi=600, transparent=True, bbox_inches="tight", format="tiff")

In [ ]:
# Plotting
# We start at 0, if you want to plot frame 2 , use 1
min_frame = 0
max_frame = 99
colormap = "inferno" # Other options: "viridis", "inferno", "plasma", "magma", "cividis"
xlabel = "Time"
ylabel = "Circle"
legendlabel = "Normalized events of TfR and Amph1 after wounding"
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

# cbar=False
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9,5), gridspec_kw={'height_ratios': [0.8, 0.1]} ) 

sns.heatmap(mean_frame_values_C1_norm_2.iloc[1:, min_frame:max_frame], vmin=0, cmap=colormap, ax=axes[0]).set_title("Clathrin")
sns.heatmap(mean_frame_values_C1_norm_2.iloc[1:, min_frame:max_frame].mean().to_numpy()[np.newaxis, :], vmin=0, cmap=colormap, ax=axes[1]).set_title("Clathrin_WholeCell")

axes[0].set_xlabel(xlabel)
axes[1].set_xlabel(xlabel)
axes[0].set_ylabel(ylabel)

fig.suptitle("Mean Particle Counts/um^2")
plt.tight_layout(h_pad=5)

# Saving figure
plt.savefig(filepath+"\\"+"ParcticleCounts_Normalized_Area_Frame.tiff", dpi=600, transparent=True, bbox_inches="tight", format="tiff")